# Data Preprocessing
1. Topic Extraction
2. User Buckets
3. Data Split

## 데이터 로드

In [84]:
import pandas as pd
COLS = ["text_tokens", "hashtags", "tweet_id", "present_media", 
                      "present_links", "present_domains", "tweet_type", "language", "tweet_timestamp", "engaged_with_user_id",
                     "engaged_with_user_follower_count", "engaged_with_user_following_count", 
                     "engaged_with_user_is_verified", "engaged_with_user_account_creation", "engaging_user_id",
                     "engaging_user_follower_count", "engaging_user_following_count", "engaging_user_is_verified",
                     "engaging_user_account_creation", "engagee_follows_engager", "reply_timestamp",
                     "retweet_timestamp", "retweet_with_comment_timestamp", "like_timestamp"]
df = pd.read_csv('../dataset/twitter/train100K.csv', names=COLS, skipinitialspace=True, skiprows=1)

In [85]:
df.head()

,text_tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,101\t10117\t140\t119\t142\t119\t152\t119\t1010...,NaN,373C0F43762B7CEC1D75728BE8A33891,NaN,A2CE3A1941BA410A1C31496C355EFCD7,E14AF8A8D257BB47587843FE7D08382B,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1582126349,2A8B6AD2B9D55F535C2441AB673133D2,...,00000865A1538142CDA5936B07FE4311,65,166,False,1452599043,False,NaN,NaN,NaN,NaN
1,101\t10105\t10817\t10124\t59232\t18121\t15629\...,NaN,773A92D9E4824D06105C02BD044BB20A,NaN,NaN,NaN,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581971193,950A95B81407F33C412E520BE55A1450,...,000009A057792FF118B9E3F2578B8407,1814,1314,False,1322868747,True,1.581979e+09,NaN,NaN,NaN
2,101\t48561\t10116\t67737\t18554\t36371\t10989\...,NaN,218A6C27871801759F7380D7C41694A6,NaN,5C683B5A29B308CADD0D7EFA7C9C32D3,6717B03E03DEE1D7ACAE37649ACA7BD6,TopLevel,9BF3403E0EB7EA8A256DA9019C0B0716,1582047119,ABB2F7F22C34057BC7B30D627B0C137A,...,00000DEF82BE9EB5CFD07FB7DB94317B,4,73,False,1573996260,False,NaN,NaN,NaN,NaN
3,101\t100055\t69940\t10414\t159\t11305\t11166\t...,NaN,AB817EBA68064A0C8CBF4A6C059D92DC,Photo,E925556EE312213AD98C4D9F131D7A8D,D722330FEBEAAE68B4F4339CE8BD7C70,TopLevel,691890251F2B9FF922BE6D3699ABEFD2,1581554925,03F96C3B7CE2179B6347AA395880C963,...,0000109A57AFA64758EE4AAE2A01BFC7,15,124,False,1385502405,True,NaN,NaN,NaN,NaN
4,101\t62154\t32221\t71843\t10143\t10237\t15507\...,NaN,349120C1E2801857530393F16D4653A5,NaN,NaN,NaN,TopLevel,9BF3403E0EB7EA8A256DA9019C0B0716,1581568955,E035DCB47CB3DF98C5CD7CFEEC3BC704,...,000012366528B5FEE179A9606DBC9826,1226,655,False,1268639592,True,1.581570e+09,NaN,NaN,NaN


***

## 버트 인코딩
- 학습가능한 인코딩을 얻기 위해서 토근화된 텍스트를 [16] BERT 모델에 input으로 넣음
- 하지만 버트 모델의 학습/추론 시간이 커서, __텍스트 정보들을 인코딩하는 새로운 피처__ 를 만들었음
- 즉, 버트를 사용해서 텍스트 인코딩 만들고, 학습 불가능한 피처로 사용하는거임
- 버트 인코딩을 사용해서 각 트윗으로부터 메인 토픽 추출 -> 한 유저에 대해(유저별로) 가장 많이나타나는 토픽들을 찾아서 결정함
- 버트의 파라미터는 업데이트되지만, 이 text token은 한번만 인풋으로 사용함
- 즉, 버트 모델을 사용해서 토큰들을 인코딩, 트윗마다 메인 topic 추출함 => 유저마다 가장 많이 보이는 5개의 topic 고르기
- 알고리즘1) 버트 모델을 사용해서 토큰들을 인코딩, 트윗마다 메인 topic 추출함
    - 버트모델로 토큰들 인코딩
    - PCA를 사용해서 분산의 95퍼센트만 유지하여 인코딩의 차원 줄임
    - k-means를 사용해서 줄인 인코딩으로부터 150개의 클러스터 생성
    - 각 트윗마다 클러스터에 topic 할당함
    
- https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python
- https://www.kaggle.com/negedng/bert-embeddings-with-tensorflow-2-0-example
- https://huggingface.co/transformers/model_doc/bert.html

In [143]:
!pip install --upgrade bert-for-tf2 params-flow sentencepiece #>> /dev/null

Requirement already up-to-date: bert-for-tf2 in /Users/ohyeji/anaconda3/lib/python3.6/site-packages (0.14.7)
Requirement already up-to-date: params-flow in /Users/ohyeji/anaconda3/lib/python3.6/site-packages (0.8.2)
  Using cached sentencepiece-0.1.94-cp36-cp36m-macosx_10_6_x86_64.whl (1.1 MB)
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.91
    Uninstalling sentencepiece-0.1.91:
      Successfully uninstalled sentencepiece-0.1.91
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

transformers 3.5.1 requires sentencepiece==0.1.91, but you'll have sentencepiece 0.1.94 which is incompatible.


In [87]:
!pip install bert-for-tf2
!pip install sentencepiece

In [88]:
from bert import tokenization
BertTokenizer = tokenization.FullTokenizer



In [98]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [99]:
BertTokenizer = bert.tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [100]:
text_tokens = df['text_tokens']

In [101]:
text_tokens.shape

(100000,)

In [105]:
!pip install tf-hub-nightly


     |████████████████████████████████| 107 kB 668 kB/s eta 0:00:01


In [106]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.3.0
Hub version:  0.10.0


In [107]:
import tensorflow_hub as hub
import tensorflow as tf
from bert.tokenization import FullTokenizer     # Still from bert module
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
import math

In [108]:
max_seq_length = 128  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

In [109]:
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])


In [110]:
# See BERT paper: https://arxiv.org/pdf/1810.04805.pdf
# And BERT implementation convert_single_example() at https://github.com/google-research/bert/blob/master/run_classifier.py

def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [111]:
# Google Colab don't need this. FullTokenizer is not updated to tf2.0 yet
tf.gfile = tf.io.gfile


In [112]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [114]:
# # s = "This is a nice sentence."
# # stokens = tokenizer.tokenize(s)

# input_ids = get_ids(stokens, tokenizer, max_seq_length)
# input_masks = get_masks(stokens, max_seq_length)
# input_segments = get_segments(stokens, max_seq_length)

# input_ids = text_tokens
# input_masks = get_masks(te)

# # print(stokens)
# print(input_ids)
# print(input_masks)
# print(input_segments)

In [115]:
def square_rooted(x):
    return math.sqrt(sum([a*a for a in x]))


def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return numerator/float(denominator)

In [116]:
pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])


NameError: name 'input_ids' is not defined

***

In [118]:

import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
from collections import namedtuple
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

TensorFlow Version: 2.3.0
Hub version:  0.10.0


In [119]:
bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=True)


In [135]:
MAX_SEQ_LEN=128
input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                    name="segment_ids")

In [122]:
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

In [124]:
FullTokenizer=bert.tokenization.FullTokenizer

vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()

do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()

tokenizer=FullTokenizer(vocab_file,do_lower_case)

def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [128]:
train_sentences = df["text_tokens"].fillna("CVxTz").values
print(train_sentences)
# list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
list_classes = range(150)
# train_y = df[list_classes].values
train_y = list_classes


['101\t10117\t140\t119\t142\t119\t152\t119\t10108\t138\t37615\t13969\t117\t10105\t100986\t86153\t24166\t17155\t10105\t36474\t34582\t10251\t18421\t118\t11135\t97992\t72894\t117\t34208\t11850\t169\t16624\t10908\t118\t32684\t10162\t100986\t10343\t118\t11735\t63803\t10115\t117\t11284\t10238\t10114\t108672\t10189\t169\t18064\t56162\t10108\t10380\t12097\t14191\t32941\t10107\t10944\t10347\t77298\t10162\t10155\t12864\t64502\t119\t14120\t131\t120\t120\t188\t119\t11170\t120\t139\t11305\t10410\t11305\t10123\t11403\t11011\t11274\t10107\t11011\t102'
 '101\t10105\t10817\t10124\t59232\t18121\t15629\t119\t119\t102'
 '101\t48561\t10116\t67737\t18554\t36371\t10989\t31288\t117\t19604\t13996\t19455\t37605\t70427\t35794\t119\t29005\t61601\t17843\t19455\t37058\t17640\t49770\t10588\t10483\t53522\t19604\t10676\t34743\t14089\t12591\t48973\t56901\t10115\t10120\t35181\t35639\t23607\t119\t14120\t131\t120\t120\t188\t119\t11170\t120\t124\t15417\t15417\t10729\t10113\t12396\t11447\t11305\t10107\t10858\t102'
 ...
 '10

In [131]:
def create_single_input(sentence,MAX_LEN):
  
    stokens = tokenizer.tokenize(sentence)

    stokens = stokens[:MAX_LEN]

    stokens = ["[CLS]"] + stokens + ["[SEP]"]

    ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
    masks = get_masks(stokens, MAX_SEQ_LEN)
    segments = get_segments(stokens, MAX_SEQ_LEN)

    return ids,masks,segments

def create_input_array(sentences):

    input_ids, input_masks, input_segments = [], [], []

    for sentence in tqdm(sentences,position=0, leave=True):

        ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2)

        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

***

In [140]:
!pip install transformers
from transformers import BertModel, BertConfig


     |████████████████████████████████| 1.3 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 1.5 MB/s eta 0:00:01
  Using cached sentencepiece-0.1.91-cp36-cp36m-macosx_10_6_x86_64.whl (1.1 MB)
Processing /Users/ohyeji/Library/Caches/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2/sacremoses-0.0.43-py3-none-any.whl
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.94
    Uninstalling sentencepiece-0.1.94:
      Successfully uninstalled sentencepiece-0.1.94


In [141]:
configuration = BertConfig()

In [147]:
model = BertModel(configuration)

ImportError: 
BertModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
